# Calculating CTR curve with Python

This notebook contains two different Methods for calculating the CTR curve, but there are some things in both of the ways to do. The next few cells are the same things for both solutions.

In [ ]:
'''Calculating CTR curve with Python'''
# Importing needed libraries for our process
import os
import json
import pandas as pd
import plotly.express as px
import plotly.io as pio
import kaleido

In [ ]:
# Creating plot images output folder
if not os.path.exists('./output plot images'):
    os.mkdir('./output plot images')

In [ ]:
# Setting width and height of the output plot images
pio.kaleido.scope.default_height = 800
pio.kaleido.scope.default_width = 2000

## First way: Calculate CTR curve for whole website or a specific URL property

In [ ]:
''' The best practice for the folder structure of our project is to have a 'data' folder, and put all of our data here.
Here for ease of doing, I didn't do this.
'''

query_df = pd.read_csv('./downloaded_data.csv')

In [ ]:
# Labeling queries position from 1 to 10
for i in range(1, 11):
    query_df.loc[(query_df['Average Position'] >= i) & (
        query_df['Average Position'] < i + 1), 'position label'] = i

In [ ]:
# Creating a grouped DataFrame based on the 'Pos label' column
query_grouped_df = query_df.groupby(['position label'])

In [ ]:
# Creating a DataFrame for storing 'query_df' manipulated data
modified_df = pd.DataFrame()

# A list for saving each position mean for our bar chart
mean_ctr_list = []

In [ ]:
''' Looping over 'query_grouped_df' groups and append the top 20% queries based on the impressions to the 'modified_df' DataFrame.
If calculating CTR only based on the top 20% of queries having the most impressions is not the best for you, you can change it.
For changing it, you can increase or decrease it by manipulating '.quantile(q=your_optimal_number, interpolation='lower')]'.
'you_optimal_number' must be between 0 to 1.
For example, if you want to get the top 30% of your queries, 'your_optimal_num' is the difference between 1 and ‌0.3 (0.7).
'''

for i in range(1, 11):
    tmp_df = query_grouped_df.get_group(i) \
        [query_grouped_df.get_group(i)['Impressions'] >= query_grouped_df.get_group(i)['Impressions']
        .quantile(q=0.8, interpolation='lower')]
    mean_ctr_list.append(tmp_df['Site CTR'].mean())
    modified_df = modified_df.append(tmp_df, ignore_index=True)

# Deleting 'tmp_df' DataFrame for reducing memory usage
del [tmp_df]

In [ ]:
# 1. The box plot
box_fig = px.box(
    modified_df,
    x='position label',
    y='Site CTR',
    title='Queries CTR distribution based on position',
    points='all',
    color='position label',
    labels={'position label': 'Position', 'Site CTR': 'CTR'})

# Showing all ten x-axes ticks
box_fig.update_xaxes(tickvals=list(range(1, 11)))

# Changing the y-axes tick format to percentage
box_fig.update_yaxes(tickformat=".0%")

# Saving plot to 'output plot images' directory
box_fig.write_image('./output plot images/Queries box plot CTR curve.png')

In [ ]:
# 2. The bar plot
bar_fig = px.bar(
    x=list(range(1, 11)),
    y=mean_ctr_list,
    title='Queries mean CTR distribution based on position',
    labels={'x': 'Position', 'y': 'CTR'},
    text_auto=True)

# Showing all ten x-axes ticks
bar_fig.update_xaxes(tickvals=list(range(1, 11)))

# Changing the y-axes tick format to percentage
bar_fig.update_yaxes(tickformat='.0%')

# Saving plot to 'output plot images' directory
bar_fig.write_image('./output plot images/Queries bar plot CTR curve.png')

## Second way: Calculating CTR curve based on landing pages URL for each directory

In [ ]:
# Creating a DataFrame for the downloaded data
with open('./downloaded_data.json') as json_file:
    landings_data = json.loads(json_file.read())['rows']
    landings_df = pd.DataFrame(landings_data)

# Renaming 'keys' column to 'landing page' column and converting 'landing page' list to an URL
landings_df.rename(columns={'keys': 'landing page'}, inplace=True)
landings_df['landing page'] = landings_df['landing page'].apply(lambda x: x[0])

# If you have a CSV file for your landing pages, you can uncomment the line below on comment the second cell line and comment above lines
# landings_df = pd.read_csv('./downloaded_data.csv')

In [ ]:
# Defining your site name between quotes. For example, 'https://www.example.com/' or 'http://mydomain.com/'
SITE_NAME = ''

In [ ]:
# Getting each landing page (URL) directory
landings_df['directory'] = landings_df['landing page'].str.extract(pat=f'((?<={SITE_NAME})[^/]+)')

In [ ]:
# For getting all directories in the output, we need to manipulate Pandas options
pd.set_option("display.max_rows", None)

# Website directories
landings_df['directory'].value_counts()

In [ ]:
''' Choose which directories are important for getting their CTR curve.
Insert the directories into the 'important_directories' variable. 
For example, 'product,tag,product-category,mag'. Separate directory values with comma.
'''

IMPORTANT_DIRECTORIES = ''
IMPORTANT_DIRECTORIES = IMPORTANT_DIRECTORIES.split(',')

In [ ]:
# Labeling landing pages position
for i in range(1, 11):
    landings_df.loc[(landings_df['position'] >= i) & (
        landings_df['position'] < i + 1), 'position label'] = i

In [ ]:
# Grouping landing pages based on their 'directory' value
landings_grouped_df = landings_df.groupby(['directory'])

In [ ]:
def each_dir_plot(dir_df, key):
    '''The function for creating and saving each directory charts'''
    # Grouping directory landing pages based on their 'position label' value
    dir_grouped_df = dir_df.groupby(['position label'])

    # Creating a DataFrame for storing 'dir_grouped_df' manipulated data
    modified_dataframe = pd.DataFrame()

    # A list for saving each position mean for our bar chart
    mean_ctr_list = []

    # Looping over 'query_grouped_df' groups and append the top 20% queries based on the impressions to the 'modified_dataframe' DataFrame.
    # If calculating CTR only based on the top 20% of queries having the most impressions is not the best for you, you can change it.
    # For changing it, you can increase or decrease it by manipulating '.quantile(q=your_optimal_number, interpolation='lower')]'.
    # 'you_optimal_number' must be between 0 to 1.
    # For example, if you want to get the top 30% of your queries, 'your_optimal_num' is the difference between 1 and ‌0.3 (0.7).

    for t in range(1, 11):
        tmp_df = dir_grouped_df.get_group(t) \
            [dir_grouped_df.get_group(t)['impressions'] >= dir_grouped_df.get_group(t)['impressions']
            .quantile(q=0.8, interpolation='lower')]
        mean_ctr_list.append(tmp_df['ctr'].mean())
        modified_dataframe = modified_dataframe.append(tmp_df, ignore_index=True)

    # 1. The box plot
    box_fig = px.box(modified_dataframe,
    x='position label',
    y='ctr',
    title=f'{key} directory CTR distribution based on position',
    points='all',
    color='position label',
    labels={'position label': 'Position', 'ctr': 'CTR'})

    # Showing all ten x-axes ticks
    box_fig.update_xaxes(tickvals=list(range(1, 11)))

    # Changing the y-axes tick format to percentage
    box_fig.update_yaxes(tickformat=".0%")

    # Saving plot to 'output plot images' directory
    box_fig.write_image(f'./output plot images/{key} directory-Box plot CTR curve.png')

    # 2. The bar plot
    bar_fig = px.bar(x=list(range(1, 11)),
    y=mean_ctr_list,
    title=f'{key} directory mean CTR distribution based on position',
    labels={'x': 'Position', 'y': 'CTR'},
    text_auto=True)

    # Showing all ten x-axes ticks
    bar_fig.update_xaxes(tickvals=list(range(1, 11)))

    # Changing the y-axes tick format to percentage
    bar_fig.update_yaxes(tickformat='.0%')

    # Saving plot to 'output plot images' directory
    bar_fig.write_image(f'./output plot images/{key} directory-Bar plot CTR curve.png')

In [ ]:
# Looping over directories and executing the 'each_dir_plot' function
for name, group in landings_grouped_df:
    if name in IMPORTANT_DIRECTORIES:
        each_dir_plot(group, name)